In [ ]:
import numpy as np
import rasterio
import numpy as np
import glob, os
import cv2
import zipfile
import tempfile
from tqdm import tqdm
from random import shuffle
%pylab inline
from pyproj import Geod
import shutil

METERS_PER_PIXEL = 50
MAX_LAT = 85
LAT_DISTANCE = 300
MIN_DROP = 500
ROTATE_ANGLE_DEGREES = 30
IMAGE_SAVE_PAD = 200
MIN_SPACING_METERS = 1000

images_dir = "/home/batman/Documents/cliffs_found/"
shutil.rmtree(images_dir, ignore_errors=True)
os.makedirs(images_dir, exist_ok=True)

def pad_array(array, num_rows, num_cols):
    padded_array = np.pad(array, ((num_rows, num_rows), (num_cols, num_cols)), mode='constant', constant_values=0)
    return padded_array

def calculate_distance(lat1, lon1, lat2, lon2):
    geod = Geod(ellps='WGS84')
    _, _, dist = geod.inv(lon1, lat1, lon2, lat2)
    return dist

def rotate(image, angle, center = None, scale = 1.0):
    (h, w) = image.shape[:2]

    if center is None:
        center = (w / 2, h / 2)

    # Perform the rotation
    M = cv2.getRotationMatrix2D(center, angle, scale)
    rotated = cv2.warpAffine(image, M, (w, h), borderValue=np.nan)

    return rotated

def find_max_diff(file_path):
  with rasterio.open(file_path) as src:
    array = src.read(1)
    bounds = src.bounds

    # High latitudes have problems
    if abs(bounds.top) > MAX_LAT:
      return None
    w_meters = calculate_distance(bounds.bottom, bounds.left, bounds.bottom, bounds.right)
    h_meters = calculate_distance(bounds.bottom, bounds.left, bounds.top, bounds.left)
    width_pixels = int(w_meters / METERS_PER_PIXEL)
    height_pixels = int(h_meters / METERS_PER_PIXEL)
    array = cv2.resize(array, (width_pixels, height_pixels), interpolation=cv2.INTER_LINEAR)

    grid = np.meshgrid(np.linspace(bounds.top, bounds.bottom, array.shape[0], dtype=np.float32),
                       np.linspace(bounds.left, bounds.right, array.shape[1], dtype=np.float32), indexing='ij')
    coord_array = np.stack(grid, axis=-1)

    desired_size = max(width_pixels, height_pixels) * 1.45
    pad_rows = int((desired_size - height_pixels) / 2)
    pad_cols = int((desired_size - width_pixels) / 2)
    array = np.pad(array, ((pad_rows, pad_rows), (pad_cols, pad_cols)), mode='constant', constant_values=np.nan)
    coord_array = np.pad(coord_array, ((pad_rows, pad_rows), (pad_cols, pad_cols), (0,0)), mode='constant', constant_values=np.nan)


  angle = 0
  qualifying_cliffs = []
  for _ in range(3):
  
  
    LAT_PIXELS = int(LAT_DISTANCE / METERS_PER_PIXEL)
    diff = np.column_stack([0.0*array[:,:LAT_PIXELS//2], array[:,LAT_PIXELS:] - array[:,:-LAT_PIXELS], 0.0*array[:,-(LAT_PIXELS-LAT_PIXELS//2):]])
    argmax_flat = np.nanargmax(diff)

    row, col = np.unravel_index(argmax_flat, diff.shape)
    drama_factor = diff[row, col]
    if drama_factor > MIN_DROP:
      lat, lon = coord_array[row, col]

      image = np.copy(array[row-IMAGE_SAVE_PAD:row+IMAGE_SAVE_PAD, col-IMAGE_SAVE_PAD:col+IMAGE_SAVE_PAD])
      unrotated_image = rotate(np.copy(image), -angle)
      qualifying_cliffs.append([drama_factor, lat, lon, angle, unrotated_image])



    array = rotate(array, ROTATE_ANGLE_DEGREES)
    coord_array = rotate(coord_array, ROTATE_ANGLE_DEGREES)
    angle += ROTATE_ANGLE_DEGREES
  

  qualifying_cliffs.sort(key=lambda x: x[0], reverse=True)
  for i, cliff in enumerate(qualifying_cliffs):
    drama_factor, lat, lon, angle, image = cliff
    
    # check if not too close to other cliffs
    too_close = False
    for other_cliff in qualifying_cliffs[:i]:
      _, other_lat, other_lon, _, _ = other_cliff
      if calculate_distance(lat, lon, other_lat, other_lon) < MIN_SPACING_METERS:
        too_close = True
    if too_close:
      continue
    else:
      print(f'Found {drama_factor}m diff at {lat}, {lon} in {file_path} at angle {angle}')
      lat_str = str(lat).replace(".", "_")
      lon_str = str(lon).replace(".", "_")
      imsave(f"{images_dir}lat_{lat_str}_lon_{lon_str}.png", np.nan_to_num(image))
      with open(f'{images_dir}full_list.txt', 'a') as the_file:
        the_file.write(f'{lat}, {lon}\n')
  return


directory = "/home/batman/Documents/global_topo_copernicus/"
all_tifs = glob.glob(directory + "*/*.tif")
shuffle(all_tifs)
for tif_file in tqdm(all_tifs):
  try:
    find_max_diff(tif_file)
  except Exception as e:
    print(f"Error processing {tif_file}: {e}")

In [ ]:
import zipfile
import tempfile
import rasterio
import matplotlib.pyplot as plt


def get_file_japan(lat, lon, get_subtile=False):
    # Calculate the bottom-left corner of the tile
    lat_bl = 5 * (lat // 5)  # Round down to the nearest multiple of 5
    lon_bl = 5 * (lon // 5)  # Round down to the nearest multiple of 5

    # Calculate the top-right corner of the tile
    lat_tr = lat_bl + 5
    lon_tr = lon_bl + 5

    # Format latitude and longitude for the tile name
    lat_bl_str = f"S{int(abs(lat_bl)):03d}" if lat_bl < 0 else f"N{int(lat_bl):03d}"
    lon_bl_str = f"W{int(abs(lon_bl)):03d}" if lon_bl < 0 else f"E{int(lon_bl):03d}"
    lat_tr_str = f"S{int(abs(lat_tr)):03d}" if lat_tr < 0 else f"N{int(lat_tr):03d}"
    lon_tr_str = f"W{int(abs(lon_tr)):03d}" if lon_tr < 0 else f"E{int(lon_tr):03d}"

    # For subtile, the tile is 1-degree by 1-degree
    lat_round = 1 * (lat // 1)  # Round down to the nearest degree
    lon_round = 1 * (lon // 1)  # Round down to the nearest degree

    # Format subtile latitude and longitude
    lat_sub_str = f"N{int(abs(lat_round)):03d}" if lat_round >= 0 else f"S{int(abs(lat_round)):03d}"
    lon_sub_str = f"E{int(abs(lon_round)):03d}" if lon_round >= 0 else f"W{int(abs(lon_round)):03d}"

    # Subtile name with prefix and suffix
    subtile_name = f"ALPSMLC30_{lat_sub_str}{lon_sub_str}_DSM"
    #subtile_name = f"ALPSMLC30_{lat_sub_str}{lon_sub_str}_MSK"
    # Construct the main tile name
    tile_name = f"{lat_bl_str}{lon_bl_str}_{lat_tr_str}{lon_tr_str}"

    return tile_name, subtile_name


def get_file_(lat, lon):
    # Format the latitude and longitude for the tile name
    lat_dir = 'N' if lat >= 0 else 'S'
    lon_dir = 'E' if lon >= 0 else 'W'
    
    # Round down to the nearest degree
    lat = int(abs(lat // 1))
    lon = int(abs(lon // 1))
    
    # Format to two digits for latitude and three for longitude
    lat_str = f"{lat_dir}{lat:02d}"
    lon_str = f"{lon_dir}{lon:03d}"
    
    # Return the formatted tile name
    #return f"ASTGTMV003_{lat_str}{lon_str}"
    name = f"Copernicus_DSM_COG_10_{lat_str}_00_{lon_str}_00"
    return f'{name}_DEM/AUXFILES/{name}_EDM.tif'
    #return f'{name}_DEM/{name}_DEM.tif'




def show_subtile(lat, lon):
    #tile_name, subtile_name = get_file_japan(lat, lon)
    tile_name = get_file_(lat, lon)
    print(tile_name)
    directory = "/home/batman/Documents/global_topo_copernicus/"
    #directory = "/home/batman/Documents/global_topo_nasa/"
    #directory = "/home/batman/Documents/alos_v4/"

    tif_file = directory + tile_name
    print(tif_file)
    #tif_file = '/home/batman/Documents/global_topo_shuttle/here.tif'
    with rasterio.open(tif_file) as src:
        # Read the data from the first band
        data = src.read(1)
        print(data[-1])
        #data = (data & 0x03) == 2
        
        # Get the bounds of the raster
        bounds = src.bounds
        
        # Get the affine transformation for the raster
        transform = src.transform
    
        # Calculate the longitude and latitude coordinates for the corners of the raster
        left, bottom, right, top = bounds.left, bounds.bottom, bounds.right, bounds.top
        longitude = [left, right, right, left, left]
        latitude = [bottom, bottom, top, top, bottom]
    
        # Transform corner coordinates to centroid coordinates of pixels
        lon, lat = rasterio.transform.xy(transform, [0, src.height-1, src.height-1, 0, 0], [0, 0, src.width-1, src.width-1, 0], offset='center')
    
    # Plotting the data with geographic coordinates
    fig, ax = plt.subplots(figsize=(10, 10))
    cax = ax.imshow(data, cmap='viridis', extent=[min(lon), max(lon), min(lat), max(lat)])
    ax.set_title('Raster Data with Geographic Coordinates')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    fig.colorbar(cax, ax=ax, label='Data values')
    plt.show()

show_subtile(-33.783952020202015, -80.80423245614034)

In [ ]:
import mpld3
mpld3.enable_notebook()

In [ ]:
import zipfile
import tempfile
import rasterio
import matplotlib.pyplot as plt

def find_tile(lat, lon):
    # Format the latitude and longitude for the tile name
    lat_dir = 'N' if lat >= 0 else 'S'
    lon_dir = 'E' if lon >= 0 else 'W'
    
    # Round down to the nearest degree
    lat = int(abs(lat // 1))
    lon = int(abs(lon // 1))
    
    # Format to two digits for latitude and three for longitude
    lat_str = f"{lat_dir}{lat:02d}"
    lon_str = f"{lon_dir}{lon:03d}"
    
    # Return the formatted tile name
    return f"ASTGTMV003_{lat_str}{lon_str}"


def show_subtile(lat, lon):
    tile_name = find_tile(lat, lon)
    print(tile_name)
    directory = "/home/batman/Documents/global_topo_nasa/"

    zip_file = directory + tile_name + '.zip'
    with tempfile.TemporaryDirectory(dir='/tmp') as temp_dir:
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(temp_dir)
            print(glob.glob(temp_dir+'/*'))
            tif_file = f'{temp_dir}/{tile_name}_dem.tif'
            print(tif_file)
        
            with rasterio.open(tif_file) as src:
                # Read the data from the first band
                data = src.read(1)
                print(data.dtype)
                print(np.max(data))
                
                # Get the bounds of the raster
                bounds = src.bounds
                
                # Get the affine transformation for the raster
                transform = src.transform
            
                # Calculate the longitude and latitude coordinates for the corners of the raster
                left, bottom, right, top = bounds.left, bounds.bottom, bounds.right, bounds.top
                longitude = [left, right, right, left, left]
                latitude = [bottom, bottom, top, top, bottom]
            
                # Transform corner coordinates to centroid coordinates of pixels
                lon, lat = rasterio.transform.xy(transform, [0, src.height-1, src.height-1, 0, 0], [0, 0, src.width-1, src.width-1, 0], offset='center')
            
            # Plotting the data with geographic coordinates
            fig, ax = plt.subplots(figsize=(10, 10))
            cax = ax.imshow(data, cmap='viridis', extent=[min(lon), max(lon), min(lat), max(lat)])
            ax.set_title('Raster Data with Geographic Coordinates')
            ax.set_xlabel('Longitude')
            ax.set_ylabel('Latitude')
            fig.colorbar(cax, ax=ax, label='Data values')
            plt.show()

# Example use of the function
import mpld3

mpld3.enable_notebook()
show_subtile(-3.7341666666666664, 121.16388888888878)

In [ ]:
from geopy.distance import geodesic

# New York coordinates
new_york = (40.0, 0.0)

# London coordinates
london = (40.0, 1.0)

# Calculate the distance
distance = geodesic(new_york, london).kilometers
print(f"The distance between New York and London is {distance:.2f} kilometers.")

In [ ]:
import rasterio

# Path to your raster file

# Open the raster file
with rasterio.open(file_path) as src:
    # Get the affine transform for the raster
    transform = src.transform

    # Get the dimensions of the raster
    width = src.width
    height = src.height

    # Create arrays to hold the longitude and latitude
    longitude, latitude = np.meshgrid(np.arange(width), np.arange(height))

    # Apply the transform to get geographic coordinates
    lon, lat = rasterio.transform.xy(transform, latitude, longitude, offset='center')

    lon = np.array(lon)
    lat = np.array(lat)

    # lon and lat are now arrays containing the longitude and latitude of each pixel
    print("Longitude array:")
    print(lon.shape)
    print("Latitude array:")
    print(lat.shape)

In [ ]:
plot()

In [ ]:
import rasterio
from rasterio.plot import show
from rasterio.windows import Window

with rasterio.open(file_path) as img:
    data = img.read(window=Window(0, 0, 100000, 100000))
    show(data)

In [ ]:
import rasterio
import matplotlib.pyplot as plt

# Open the raster file
file_path = '/home/batman/Documents/ASTGTMV003_N68W029_dem.tif'
with rasterio.open(file_path) as src:
    # Read the data from the first band
    data = src.read(1)
    
    # Get the bounds of the raster
    bounds = src.bounds
    
    # Get the affine transformation for the raster
    transform = src.transform

    # Calculate the longitude and latitude coordinates for the corners of the raster
    left, bottom, right, top = bounds.left, bounds.bottom, bounds.right, bounds.top
    longitude = [left, right, right, left, left]
    latitude = [bottom, bottom, top, top, bottom]

    # Transform corner coordinates to centroid coordinates of pixels
    lon, lat = rasterio.transform.xy(transform, [0, src.height-1, src.height-1, 0, 0], [0, 0, src.width-1, src.width-1, 0], offset='center')

# Plotting the data with geographic coordinates
fig, ax = plt.subplots(figsize=(10, 10))
cax = ax.imshow(data, cmap='viridis', extent=[min(lon), max(lon), min(lat), max(lat)])
ax.set_title('Raster Data with Geographic Coordinates')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
#ylim(16.4, 16.8)
#xlim(17, 17.4)
fig.colorbar(cax, ax=ax, label='Data values')
plt.show()

In [ ]:
print(min(array.flatten()))

In [ ]:
import seaborn as sns

In [ ]:
sns.distplot(array.flatten()[::100])